# Hybrid Search

Baseline2の文書検索部分をハイブリッド検索にして試す

In [1]:
import os
import pandas as pd
import openai
import datetime
import tiktoken
from sudachipy import tokenizer
from sudachipy import dictionary
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [2]:
# OpenAI APIキーを設定
openai.api_key = input()

In [3]:
novel_file_path = "../data/novels_preprocess/works/"

## テキス読み込み、リスト化

In [4]:
# .txt ファイルを読み込み、ドキュメントをリスト化
documents = []
for filename in os.listdir(novel_file_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(novel_file_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        documents.extend(loader.load())

In [1]:
documents

In [2]:
# テキストを分割するためのテキストスプリッターを定義
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)
page_contents = [doc.page_content for doc in split_docs]

In [3]:
for i in range(len(split_docs)):
    print(split_docs[i].page_content)

## FAISSでベクトル検索構築

In [8]:
# OpenAIの埋め込みモデルを使ってドキュメントをベクトル化
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_41892\900527064.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [9]:
# FAISSでベクトルストアを作成
vectorstore = FAISS.from_documents(split_docs, embedding)

In [10]:
# Retrieverとしてベクトルストアを設定し、top_kを設定
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

## キーワード検索構築

In [11]:
# SudachiPyを使用して名詞・動詞・形容詞のみを抽出する関数
def extract_relevant_words(query):

    tokenizer_obj = dictionary.Dictionary(dict="full").create()
    mode = tokenizer.Tokenizer.SplitMode.C  # モードを指定
    tokens = tokenizer_obj.tokenize(query, mode)
    
    # 名詞、動詞、形容詞のみを抽出
    relevant_words = []
    for token in tokens:
        pos = token.part_of_speech()[0]  # 品詞情報を取得
        if pos in ["名詞", "動詞", "形容詞"]:
            relevant_words.append(token.surface())  # 単語の表層形を取得
    relevant_words = list(set(relevant_words))  # 重複削除
    
    return " ".join(relevant_words)

## ハイブリッド検索

In [12]:
# Retrieverの準備
bm25_retriever = BM25Retriever.from_texts(
    page_contents, 
    preprocess_func=extract_relevant_words,
    k=5,
)

In [13]:
# 6. 回答を50トークン以内に制限し、引用を含むプロンプトを作成
prompt_template = """あなたは正確性の高いQAシステムです。
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
以下のルールに従って回答してください。:
1. 事前知識は使わず、コンテキストから得られる情報のみを使用して回答してください。
2. 回答内で指定されたコンテキストを直接参照しないでください。
3. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。
4. 回答は50トークン以内で簡潔に回答してください。
5. コンテキストから具体的な回答ができない場合は「分かりません」と回答してください。

コンテキスト: {context}
質問: {question}
回答:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


In [14]:
# OpenAIの言語モデルを設定（ここではGPT-3を使用）
llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_41892\1591884372.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)


In [15]:
# Retrieverの準備
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], 
    weights=[0.5, 0.5]
)

In [16]:
# 検索用のQAチェーンを構築
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" モードはシンプルに関連ドキュメントをまとめて渡すモード
    retriever=ensemble_retriever,
    return_source_documents=True,  # 検索結果としてソースドキュメントを返す
    chain_type_kwargs={"prompt": prompt}
)

## 質問ファイルを読み込んでQ&Aを作成

In [17]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../data/query.csv", encoding="utf-8")

In [4]:
query_df.head()

In [5]:
answers = []
evidences = []

for _, row in query_df.iterrows():
    print(_)
    query = row["problem"]
    processed_query = extract_relevant_words(query)  # 質問を前処理して名詞・動詞・形容詞だけを抽出
    print(query)
    print(processed_query)
    result = qa_chain({"query": processed_query})
    answer = result["result"]
    print(answer)
    evidence = result["source_documents"][0].page_content # 証拠部分を抽出
    answers.append(answer)
    evidences.append(evidence)

In [20]:
# DataFrameに回答と証拠を追加
query_df['full_answer'] = answers
query_df['evidence'] = evidences

In [21]:
replace_dict = {
        "\n": "",
        "\r": "",
    }

query_df = query_df.replace(
        {"evidence": replace_dict},
        regex=True
    )

In [6]:
query_df.head(20)

In [29]:
# LLMを使って要約を行う関数
def summarize_answer(problem: str, full_answer: str, evidence: str) -> str:

    summarize_prompt = PromptTemplate(
        input_variables=["problem", "full_answer", "evidence"],
        template=
            """以下のQuestionに対するAnswerの文章をEvidenceを元に50文字以内に収まるように簡潔に答え直してください。
            回答だけを答えてください。
                f"Question: {problem}\n\n"
                f"Answer: {full_answer}\n"
                f"Evidence: {evidence}\n"
            回答:"""
    )
    chain = summarize_prompt | llm

    response = chain.invoke(
        {"problem": problem, "full_answer": full_answer, "evidence": evidence}
    )
    return response.content

In [30]:
# tiktokenとgpt-4のトークナイザーを取得
enc = tiktoken.encoding_for_model("gpt-4-2024-08-06")

# query_df の "answer" 列のトークン数を計算し、50トークンを超える場合は要約を行う関数
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    def summarize_if_needed(problem: str, full_answer: str, evidence: str) -> str:
        # トークン数を計算
        token_count = len(enc.encode(full_answer))
        print(token_count)
        
        # トークン数が50を超えた場合は要約する
        if token_count > 50:
            # LLMを使って要約
            summarized_answer = summarize_answer(problem, full_answer, evidence)
            return summarized_answer
        return full_answer

    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["full_answer"]
    for i in range(len(query_df.index)):
        query_df["answer"][i] = summarize_if_needed(query_df["problem"][i], query_df["full_answer"][i], query_df["evidence"][i])
    return query_df

In [7]:
query_df = check_and_summarize_answers(query_df)

In [8]:
query_df.head(20)

In [33]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [34]:
# backup
dt_now = datetime.datetime.now()
ymdm = dt_now.strftime("%Y%m%d-%H%M")

query_df[['index', 'problem', 'full_answer', 'answer', 'evidence']].to_csv(
    f"../submit/{ymdm}_predictions.csv",
    index=False,
    header=True,
    encoding="utf-8-sig"
)